In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import scanpy as sc
import warnings
import diptest
warnings.filterwarnings('ignore')

In [29]:
class CellHeterogeneityAnalyzer:
    def __init__(self, expression_matrix, cell_types, gene_names=None):
        """
        Initialize the analyzer with expression data and cell type labels
        
        Parameters:
        -----------
        expression_matrix : np.ndarray
            Genes x Cells matrix of expression values
        cell_types : np.array
            Array of cell type labels for each cell
        gene_names : np.array, optional
            Array of gene names corresponding to rows in expression_matrix
        """
        # Ensure data is numpy array
        self.expression = np.array(expression_matrix)
        self.cell_types = np.array(cell_types)
        self.gene_names = gene_names if gene_names is not None else np.arange(expression_matrix.shape[0])
        
    def calculate_transcriptional_entropy(self, cell_type=None):
        """Calculate Shannon entropy of gene expression for each cell"""
        # Get relevant subset of data
        if cell_type is not None:
            mask = self.cell_types == cell_type
            exp_subset = self.expression[:, mask]
        else:
            exp_subset = self.expression
            
        # Ensure non-negative values and add small constant
        exp_subset = np.maximum(exp_subset, 0) + 1e-10
        
        # Normalize each cell (column) to sum to 1
        col_sums = np.sum(exp_subset, axis=0)
        col_sums = np.where(col_sums == 0, 1, col_sums)  # Avoid division by zero
        exp_norm = exp_subset / col_sums[np.newaxis, :]
        
        # Calculate entropy safely
        entropy = -np.sum(exp_norm * np.log2(exp_norm), axis=0)
        
        # Replace any remaining NaN values with 0
        entropy = np.nan_to_num(entropy, 0)
        
        return entropy

    def calculate_activation_score(self, activation_genes, cell_type=None):
        """Calculate per-cell activation score based on mean expression of activation markers"""
        if isinstance(activation_genes[0], str) and self.gene_names is not None:
            gene_indices = [i for i, gene in enumerate(self.gene_names) if gene in activation_genes]
        else:
            gene_indices = activation_genes
            
        if not gene_indices:
            return np.array([])
            
        activation_exp = self.expression[gene_indices]
        activation_score = np.mean(activation_exp, axis=0)
        
        if cell_type is not None:
            mask = self.cell_types == cell_type
            activation_score = activation_score[mask]
            
        return activation_score
    
    def calculate_gene_variability(self, cell_type=None):
        """Calculate coefficient of variation for each gene within cell types"""
        if cell_type is not None:
            mask = self.cell_types == cell_type
            exp_subset = self.expression[:, mask]
        else:
            exp_subset = self.expression
            
        means = np.mean(exp_subset, axis=1) + 1e-10
        stds = np.std(exp_subset, axis=1)
        cv = stds / means
        
        return pd.Series(cv, index=self.gene_names)
    
    def find_bimodal_genes(self, cell_type=None, significance=0.05, min_cells=10):
        """
        Identify genes showing bimodal expression distribution
        """
        if cell_type is not None:
            mask = self.cell_types == cell_type
            exp_subset = self.expression[:, mask]
        else:
            exp_subset = self.expression
            
        # Skip analysis if too few cells
        if exp_subset.shape[1] < min_cells:
            return pd.Series()
            
        bimodal_genes = []
        p_values = []
        
        for i, gene in enumerate(self.gene_names):
            gene_expr = exp_subset[i]
            
            # Skip genes with no or very low variance
            if np.var(gene_expr) < 1e-10:
                continue
                
            # Perform Hartigan's dip test
            try:
                _, p_val = diptest.diptest(gene_expr)
                if p_val < significance:
                    bimodal_genes.append(gene)
                    p_values.append(p_val)
            except:
                continue
                
        return pd.Series(p_values, index=bimodal_genes)
    
    def analyze_cell_type_heterogeneity(self, cell_type, activation_genes=None):
        """
        Comprehensive analysis of heterogeneity for a specific cell type
        """
        mask = self.cell_types == cell_type
        if np.sum(mask) < 1:
            return {'error': f'No cells found for type {cell_type}'}
            
        results = {}
        
        # Basic metrics
        results['cell_count'] = np.sum(mask)
        
        # Transcriptional heterogeneity
        entropy = self.calculate_transcriptional_entropy(cell_type)
        results['entropy_mean'] = np.mean(entropy)
        results['entropy_std'] = np.std(entropy)
        
        # Gene variability
        cv = self.calculate_gene_variability(cell_type)
        results['top_variable_genes'] = cv.nlargest(10)
        
        # Bimodal genes
        bimodal = self.find_bimodal_genes(cell_type)
        results['bimodal_genes'] = bimodal
        
        # Activation score if activation genes provided
        if activation_genes:
            act_scores = self.calculate_activation_score(activation_genes, cell_type)
            if len(act_scores) > 0:
                results['activation_score_mean'] = np.mean(act_scores)
                results['activation_score_std'] = np.std(act_scores)
            
        return results

def analyze_single_population(analyzer, cell_type, activation_markers=None):
    """Analyze a single cell population"""
    
    results = analyzer.analyze_cell_type_heterogeneity(cell_type, activation_markers)
    
    if isinstance(results, dict) and 'error' in results:
        return results['error']
    
    # Add bimodality analysis
    bimodal_genes = analyzer.find_bimodal_genes(cell_type)
    if not bimodal_genes.empty:
        results['bimodal_genes'] = {
            'genes': bimodal_genes.index.tolist(),
            'p_values': bimodal_genes.values.tolist()
        }
        
        # Print top bimodal genes
        print(f"\nTop bimodal genes for {cell_type}:")
        for gene, p_val in bimodal_genes.nsmallest(10).items():
            print(f"  {gene}: p-value = {p_val:.6f}")
    
    return results

def analyze_immune_populations(analyzer, immune_cells, activation_markers=None):
    """Analyze multiple immune cell populations"""
    results = {}

    # Get total number of cells
    total_cells = analyzer.cell_types.size
    
    # Calculate total immune cells
    total_immune_cells = sum(np.sum(analyzer.cell_types == cell_type) 
                           for cell_type in immune_cells)
    
    print("\n=== Immune Cell Population Summary ===")
    print(f"Total cells in dataset: {total_cells}")
    print(f"Total immune cells: {total_immune_cells}")
    
    for cell_type in immune_cells:
        print(f"\n{'='*20} Analyzing {cell_type} {'='*20}")
        
        # Get basic population statistics
        count = np.sum(analyzer.cell_types == cell_type)
        if count == 0:
            print(f"No {cell_type} found in the dataset")
            continue
        
        # Get markers for this cell type if available
        cell_markers = None
        if activation_markers and cell_type in activation_markers:
            if 'activated' in activation_markers[cell_type]:
                cell_markers = activation_markers[cell_type]['activated']
        
        # Get detailed analysis
        cell_results = analyze_single_population(analyzer, cell_type, cell_markers)
        
        if isinstance(cell_results, str):  # Error message
            print(cell_results)
            continue
            
        # Add population percentages
        cell_results['percent_of_immune'] = float((count/total_immune_cells) * 100)
        cell_results['percent_of_total'] = float((count/total_cells) * 100)
        
        results[cell_type] = cell_results
        
        # Print comprehensive summary
        print(f"\nPopulation Statistics:")
        print(f"  Count: {count}")
        print(f"  % of immune cells: {cell_results['percent_of_immune']:.1f}%")
        print(f"  % of total cells: {cell_results['percent_of_total']:.1f}%")
        print(f"\nHeterogeneity Metrics:")
        print(f"  Transcriptional entropy: {cell_results['entropy_mean']:.3f} ± {cell_results['entropy_std']:.3f}")
        
        # Print top variable genes
        print("\nTop 5 Most Variable Genes:")
        for gene, cv in list(cell_results['top_variable_genes'].items())[:5]:
            print(f"  {gene}: CV = {cv:.3f}")
        
        # Print activation scores if available
        if 'activation_score_mean' in cell_results:
            print(f"\nActivation Score: {cell_results['activation_score_mean']:.3f} ± {cell_results['activation_score_std']:.3f}")
    
    return results

In [25]:
## Load real and generated scRNA-seq data
run_dir = "/Users/guyshani/Documents/PHD/Aim_2/test_models/run_20250201_161547_dataset_singler_label/"
data_path = "/Users/guyshani/Documents/PHD/Aim_2/10x_data_mouse/30_1_2025__normalized/"

df_gen = pd.read_csv(f'{run_dir}_generated_data.csv')   # load generated data
cell_type_gen = df_gen['cell_type']   # get cell type column
dataset = df_gen['dataset'] # get dataset column
df_gen = df_gen.drop(df_gen.columns[-2:], axis=1)   # drop label columns
df_real = pd.read_csv(f'{data_path}combined_data.csv', sep=";")  # load real data
df_real_meta = pd.read_csv(f'{data_path}combined_metadata.csv', sep=";")  # load real data metadata
cell_type_real = df_real_meta['singler_label']   # get cell type column
df_gen.columns = df_real.columns # add gene names
gene_names = df_real.columns

# process data
## replace "," with "." and convert string to float for the real data we loaded
df_real = df_real.replace(',', '.', regex=True)
df_real = df_real.apply(pd.to_numeric)
df_real = df_real.to_numpy()
df_gen = df_gen.to_numpy()
# test that both matrices have the same size
print("x_test shape:", df_real.shape)
print("x_gen shape:", df_gen.shape)

df_real_t = df_real.T
df_gen_t = df_gen.T

x_test shape: (41588, 2000)
x_gen shape: (41588, 2000)


In [ ]:
## Generated data
# Convert cell_types to a numpy array
cell_types_array_gen = np.array(cell_type_gen)  # Assuming 'cell_type' is your original array with cell labels
print("Cell types shape before:", cell_types_array_gen.shape)

# Initialize analyzer with your data
analyzer_gen = CellHeterogeneityAnalyzer(
    expression_matrix=df_gen_t,  # genes x cells matrix
    cell_types=cell_types_array_gen,
    gene_names=gene_names
)
# Verify the data
print("\nVerification Generated data:")
print("Expression matrix shape:", analyzer_gen.expression.shape)
print("Cell types shape:", analyzer_gen.cell_types.shape)
print("Gene names shape:", analyzer_gen.gene_names.shape)
print("\nUnique cell types and counts:")
unique_cells, counts = np.unique(analyzer_gen.cell_types, return_counts=True)
for cell_type, count in zip(unique_cells, counts):
    print(f"{cell_type}: {count} cells")


## Real data
# Convert cell_types to a numpy array
cell_types_array_real = np.array(cell_type_real)  # Assuming 'cell_type' is your original array with cell labels
print("Cell types shape before:", cell_types_array_real.shape)

# Initialize analyzer with your data
analyzer_real = CellHeterogeneityAnalyzer(
    expression_matrix=df_real_t,  # genes x cells matrix
    cell_types=cell_types_array_real,
    gene_names=gene_names
)
# Verify the data
print("\nVerification Real data:")
print("Expression matrix shape:", analyzer_real.expression.shape)
print("Cell types shape:", analyzer_real.cell_types.shape)
print("Gene names shape:", analyzer_real.gene_names.shape)
print("\nUnique cell types and counts:")
unique_cells, counts = np.unique(analyzer_real.cell_types, return_counts=True)
for cell_type, count in zip(unique_cells, counts):
    print(f"{cell_type}: {count} cells")

In [ ]:
activation_markers = {
    'T cells': {
        'naive': ['Ccr7', 'Sell', 'Il7r', 'Cd62l'],  # CD62L is Sell
        'activated': ['Cd69', 'Cd25', 'Cd44', 'Il2ra', 'Hla-dr'],
        'effector': ['Gzmb', 'Prf1', 'Ifng', 'Tbx21'],
        'memory': ['Il7r', 'Cd44', 'Bcl2', 'Cd127'],
        'exhausted': ['Pdcd1', 'Lag3', 'Ctla4', 'Havcr2', 'Tigit']  # PD-1 is Pdcd1, Tim-3 is Havcr2
    },
    
    'B cells': {
        'naive': ['Cd19', 'Cd79a', 'Cd79b', 'Ighd', 'Il7r'],
        'activated': ['Cd69', 'Cd86', 'Cd83', 'Cd40', 'Aicda'],
        'memory': ['Cd27', 'Cd38', 'Ighg1'],
        'plasma': ['Sdc1', 'Xbp1', 'Prdm1']  # CD138 is Sdc1, BLIMP1 is Prdm1
    },
    
    'NK cells': {
        'resting': ['Cd56', 'Ncr1', 'Klrb1', 'Il7r'],  # CD56 is Ncam1
        'activated': ['Cd69', 'Ifng', 'Gzmb', 'Prf1', 'Nkg2d'],
        'cytokine_activated': ['Il2rb', 'Cd25', 'Stat4', 'Stat5']
    },
    
    'Dendritic cells': {
        'immature': ['Cd11c', 'Cd1c', 'Cd1a'],
        'activated': ['Cd83', 'Cd86', 'Cd80', 'Il12b', 'Ccr7'],
        'tolerogenic': ['Il10', 'Tgfb1', 'Pdl1', 'Ido1']
    },
    
    'Macrophages': {
        'M0': ['Cd68', 'Adgre1', 'Cd14'],  # F4/80 is Adgre1
        'M1': ['Nos2', 'Il1b', 'Tnf', 'Cxcl10', 'Il6'],
        'M2': ['Arg1', 'Mrc1', 'Cd163', 'Il10', 'Tgfb1']
    },
    
    'Monocytes': {
        'classical': ['Cd14', 'Ccr2', 'Ly6c2'],
        'non_classical': ['Cx3cr1', 'Nr4a1', 'Cd16'],
        'activated': ['Cd69', 'Il1b', 'Tnf', 'Il6']
    },
    
    'Granulocytes': {
        'neutrophils': ['S100a8', 'S100a9', 'Csf3r', 'Cxcr2'],
        'activated_neutrophils': ['Cd69', 'Il1b', 'Mpo', 'Elane', 'Fcgr3'],
        'eosinophils': ['Il5ra', 'Siglec8', 'Prg2'],
        'basophils': ['Ms4a2', 'Il3ra', 'Fcer1a']
    },
    
    'Microglia': {
        'homeostatic': ['P2ry12', 'Tmem119', 'Cx3cr1', 'Sall1'],
        'activated': ['Cd69', 'Il1b', 'Tnf', 'Nos2', 'Trem2'],
        'disease_associated': ['Apoe', 'Trem2', 'Cst7', 'Cd74']
    }
}

# Check which markers are present in the data for each cell type
print("Available activation markers in dataset:")
available_markers = {}

for cell_type, states in activation_markers.items():
    print(f"\n=== {cell_type} ===")
    available_markers[cell_type] = {}
    
    for state, markers in states.items():
        present_markers = [gene for gene in markers if gene in gene_names]
        available_markers[cell_type][state] = present_markers
        print(f"{state}: {present_markers}")

In [30]:
# Define immune cells to analyze
immune_cells = ['B cells', 'T cells', 'NK cells', 'Dendritic cells', 
                'Granulocytes', 'Macrophages', 'Monocytes','Microglia']

# Run analysis for all immune populations
results_gen = analyze_immune_populations(analyzer_gen, immune_cells, activation_markers)
results_real = analyze_immune_populations(analyzer_real, immune_cells, activation_markers)


# If you want to analyze just one cell type:

t_cell_results_gen = analyze_single_population(
    analyzer_gen,
    'T cells',
    activation_markers['T cells']['activated'])
t_cell_results_real = analyze_single_population(
    analyzer_real,
    'T cells',
    activation_markers['T cells']['activated'])



=== Immune Cell Population Summary ===
Total cells in dataset: 41588
Total immune cells: 40723

==================== Analyzing B cells ====================

Top bimodal genes for B cells:
  Cd74: p-value = 0.000000
  H2-Aa: p-value = 0.000000
  Cd79a: p-value = 0.000000
  H2-Eb1: p-value = 0.000000
  S100a6: p-value = 0.000000
  mt-Atp8: p-value = 0.000000
  H2-Ab1: p-value = 0.000000
  Uba52: p-value = 0.000000
  Igkc: p-value = 0.000000
  Gpx1: p-value = 0.000000

Population Statistics:
  Count: 18719
  % of immune cells: 46.0%
  % of total cells: 45.0%

Heterogeneity Metrics:
  Transcriptional entropy: 10.207 ± 0.205

Top 5 Most Variable Genes:
  Stmn1: CV = 8.921
  Prkch: CV = 3.352
  Ccl5: CV = 3.220
  Il7r: CV = 3.191
  Ms4a6c: CV = 3.183

Activation Score: 1.011 ± 1.073

==================== Analyzing T cells ====================

Top bimodal genes for T cells:
  Cd74: p-value = 0.000000
  H2-Aa: p-value = 0.000000
  Cd79a: p-value = 0.000000
  H2-Eb1: p-value = 0.000000
  mt-A

In [31]:
def format_t_cell_results(results):
    # Basic metrics with formatted numbers
    basic_metrics = pd.DataFrame({
        'Metric': ['Cell Count', 'Entropy Mean', 'Entropy Std', 'Activation Score Mean', 'Activation Score Std'],
        'Value': [
            results['cell_count'],
            f"{results['entropy_mean']:.4f}",
            f"{results['entropy_std']:.4f}",
            f"{results['activation_score_mean']:.4f}",
            f"{results['activation_score_std']:.4f}"
        ]
    })
    
    # Top variable genes with formatted numbers
    var_genes = pd.DataFrame({
        'Gene': results['top_variable_genes'].index,
        'Variability Score': [f"{value:.4f}" for value in results['top_variable_genes'].values]
    })
    
    # Bimodal genes with formatted p-values
    bimodal = pd.DataFrame({
        'Gene': results['bimodal_genes']['genes'],
        'P-value': [f"{value:.4f}" for value in results['bimodal_genes']['p_values']]
    }).sort_values('P-value')  # Sort by p-value for better presentation
    
    # Create output DataFrames with sections
    dfs = []
    
    # Basic Metrics Section
    dfs.append(pd.DataFrame({'Section': ['BASIC METRICS']}))
    dfs.append(basic_metrics)
    dfs.append(pd.DataFrame({'Section': ['']}))  # Empty row
    
    # Top Variable Genes Section
    dfs.append(pd.DataFrame({'Section': ['TOP VARIABLE GENES']}))
    dfs.append(var_genes)
    dfs.append(pd.DataFrame({'Section': ['']}))  # Empty row
    
    # Top 20 Bimodal Genes Section
    dfs.append(pd.DataFrame({'Section': ['TOP 20 BIMODAL GENES']}))
    dfs.append(bimodal.head(20))
    
    # Combine all sections
    final_df = pd.concat(dfs, ignore_index=True)
    
    return final_df

def format_immune_cell_results(results):
    # Basic metrics section (keeping as is)
    all_cells_basic = []
    for cell_type, data in results.items():
        basic_metrics = {
            'Cell Type': cell_type,
            'Cell Count': data.get('cell_count', 'N/A'),
            'Entropy Mean': f"{data.get('entropy_mean', 0):.4f}",
            'Entropy Std': f"{data.get('entropy_std', 0):.4f}",
        }
        
        if 'activation_mean' in data and 'activation_std' in data:
            basic_metrics.update({
                'Activation Score Mean': f"{data['activation_mean']:.4f}",
                'Activation Score Std': f"{data['activation_std']:.4f}"
            })
        elif 'activation_score_mean' in data and 'activation_score_std' in data:
            basic_metrics.update({
                'Activation Score Mean': f"{data['activation_score_mean']:.4f}",
                'Activation Score Std': f"{data['activation_score_std']:.4f}"
            })
        
        if 'percent_of_immune' in data:
            basic_metrics['Percent of Immune'] = f"{data['percent_of_immune']:.4f}"
        if 'percent_of_total' in data:
            basic_metrics['Percent of Total'] = f"{data['percent_of_total']:.4f}"
            
        all_cells_basic.append(basic_metrics)
    
    basic_df = pd.DataFrame(all_cells_basic)
    
    # Variable genes section (restructured)
    var_genes_dict = {}
    var_scores_dict = {}
    max_var_genes = 0
    
    for cell_type, data in results.items():
        if 'top_variable_genes' in data and not data['top_variable_genes'].empty:
            genes = data['top_variable_genes'].index.tolist()
            scores = [f"{value:.4f}" for value in data['top_variable_genes'].values]
            var_genes_dict[cell_type] = genes
            var_scores_dict[cell_type] = scores
            max_var_genes = max(max_var_genes, len(genes))
    
    # Create variable genes DataFrame with cell types as columns
    var_genes_rows = []
    for i in range(max_var_genes):
        row = {'Rank': i + 1}
        for cell_type in results.keys():
            if cell_type in var_genes_dict:
                genes = var_genes_dict[cell_type]
                scores = var_scores_dict[cell_type]
                row[f"{cell_type} Gene"] = genes[i] if i < len(genes) else ""
                row[f"{cell_type} Score"] = scores[i] if i < len(scores) else ""
        var_genes_rows.append(row)
    
    # Bimodal genes section (restructured)
    bimodal_dict = {}
    max_bimodal = 0
    
    for cell_type, data in results.items():
        if ('bimodal_genes' in data and 
            not isinstance(data['bimodal_genes'], pd.Series) and 
            len(data['bimodal_genes'].get('genes', [])) > 0):
            
            genes = data['bimodal_genes']['genes']
            p_values = [f"{value:.4f}" for value in data['bimodal_genes']['p_values']]
            bimodal_dict[cell_type] = list(zip(genes, p_values))
            max_bimodal = max(max_bimodal, len(genes))
    
    # Create bimodal genes DataFrame with cell types as columns
    bimodal_rows = []
    for i in range(max_bimodal):
        row = {'Rank': i + 1}
        for cell_type in results.keys():
            if cell_type in bimodal_dict:
                gene_pvals = bimodal_dict[cell_type]
                if i < len(gene_pvals):
                    row[f"{cell_type} Gene"] = gene_pvals[i][0]
                    row[f"{cell_type} P-value"] = gene_pvals[i][1]
                else:
                    row[f"{cell_type} Gene"] = ""
                    row[f"{cell_type} P-value"] = ""
        bimodal_rows.append(row)
    
    # Combine all sections
    dfs = []
    
    # Basic Metrics Section
    dfs.append(pd.DataFrame({'Section': ['BASIC METRICS FOR ALL CELL TYPES']}))
    dfs.append(basic_df)
    dfs.append(pd.DataFrame({'Section': ['']}))
    
    # Variable Genes Section
    if var_genes_rows:
        dfs.append(pd.DataFrame({'Section': ['TOP VARIABLE GENES BY CELL TYPE']}))
        var_genes_df = pd.DataFrame(var_genes_rows)
        dfs.append(var_genes_df)
        dfs.append(pd.DataFrame({'Section': ['']}))
    
    # Bimodal Genes Section
    if bimodal_rows:
        dfs.append(pd.DataFrame({'Section': ['BIMODAL GENES BY CELL TYPE']}))
        bimodal_df = pd.DataFrame(bimodal_rows)
        dfs.append(bimodal_df)
    
    # Combine all sections
    final_df = pd.concat(dfs, ignore_index=True)
    
    return final_df


In [32]:
### Foramt and save results
formatted_results_gen = format_immune_cell_results(results_gen)
formatted_results_real = format_immune_cell_results(results_real)

# Save to CSV
formatted_results_gen.to_csv(run_dir+'immune_cell_analysis_results_generated.csv', index=False)
formatted_results_real.to_csv(run_dir+'immune_cell_analysis_results_real.csv', index=False)

'''
# Print preview of the formatted results
print("\nFirst few rows of the formatted results:")
print(formatted_results_gen.head(15))
'''

'\n# Print preview of the formatted results\nprint("\nFirst few rows of the formatted results:")\nprint(formatted_results_gen.head(15))\n'

In [33]:
# Format the results
formatted_results_gen = format_t_cell_results(t_cell_results_gen)
formatted_results_real = format_t_cell_results(t_cell_results_real)

# Save to CSV
formatted_results_gen.to_csv(run_dir+'t_cell_analysis_results_generated.csv', index=False, float_format='%.4f')
formatted_results_real.to_csv(run_dir+'t_cell_analysis_results_real.csv', index=False, float_format='%.4f')

'''
# Print preview of the formatted results
print("\nFirst few rows of the formatted results:")
print(formatted_results.head(15))
'''


'\n# Print preview of the formatted results\nprint("\nFirst few rows of the formatted results:")\nprint(formatted_results.head(15))\n'